In [1]:
import torch
import torch.nn as nn

from model_class import ResNet
from dataset_utils import get_data_loader


def test(net, batch_size, num_classes, device):
    testloader = get_data_loader('test', batch_size, num_classes, shuffle=True)


    criterion = nn.CrossEntropyLoss()

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for _, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test accuracy=%.4f" % (test_loss / (num_val_steps), val_acc))
    return val_acc  # my addition. Originally returns None


device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

out_chan_dict = {2: 8, 4: 10, 8: 12, 16: 14, 32: 16, 64: 18}
num_classes = 4

net = ResNet(num_classes, out_chan=out_chan_dict[num_classes]).to(device)
net.load_state_dict(torch.load(f"./checkpoints/resnet_{num_classes}.pt"))
print(test(net, batch_size=50, num_classes=num_classes, device=device))

Test Loss=0.2116, Test accuracy=0.9150
0.915
